In [1]:
import numpy as np
from sklearn import svm
from sklearn.linear_model import LinearRegression
import optunity.metrics as metrics
from sklearn.neural_network import MLPRegressor as mlpr
from pylab import rcParams
import pandas as pd
import data_preprocessor
import random
import copy
import matplotlib.pyplot as plt
from matplotlib import animation, rc
from IPython.display import HTML
import matplotlib
import matplotlib.animation as animation
import matplotlib.patches as patches
from pylab import rcParams

class Q_Criterion:
    def __init__(self, X_train, y_train, X_test, y_test):
        self.X_train = X_train
        self.y_train = y_train
        
        self.X_test = X_test
        self.y_test = y_test
        
    def get_list_of_indexes_by_bitmask(self, bitmask):
        ls = []
        for i in range(len(bitmask)):
            if bitmask[i] == 1:
                ls.append(i)
                
        return ls 
    
    def calc_Q(self, feature_bitmask):
        list_of_features = self.get_list_of_indexes_by_bitmask(feature_bitmask)
        
        model = LinearRegression()
        model.fit(self.X_train[:, list_of_features], self.y_train)
        
        y_hat = model.predict(self.X_test[:, list_of_features])
        Q = metrics.mse(y_hat, self.y_test)
        return Q
        
        
class Stochastic_Search:
    def __init__(self):
        return
    
    
    
    # -------------------------------------------------------------------------
    def draw(self, mode, data, additional_information = []):    
        output_plot = plt.figure(dpi = 40)
        
        output_subplot = output_plot.add_subplot(111, aspect = 'equal')
        
        area_for_arrows = 0.2
        a = (1.0-area_for_arrows)/len(data[0])
        border_width = a/10
 
        plt.text(0.4, 0.4, mode, size = 50, rotation = -10.0, color = 'white',
                 ha = "center", va = "center",
                 bbox = dict(boxstyle = "round",
                          ec = (0.1, 0.1, 0.1),
                          fc = (0.1, 0.1, 0.1)),
        )
        
        for k in range(len(data)):
            cur_col = 'black'
            if k == 1:
                cur_col = 'blue'
            if k == 2:
                cur_col = 'red'
            
            for i in range(len(data[k])):
                if len(additional_information) > 0:
                    if k == 0:
                        if additional_information[i] == 0:
                            cur_col = 'blue'
                        else:
                            cur_col = 'red'
                    
                
                output_subplot.add_patch(
                        patches.Rectangle(
                            (a*i+border_width, k*a+border_width),   # (x,y)
                            a-2*border_width,          # width
                            a-2*border_width,          # height
                            color = 'black',
                            alpha = data[k][i]
                        )
                    )
                            
                output_subplot.add_patch(
                        patches.Rectangle(
                            (a*i+border_width, k*a+border_width),   # (x,y)
                            a-2*border_width,          # width
                            a-2*border_width,          # height
                            color = cur_col,
                            fill = False,
                            linewidth = 4
                        )
                    )
                        
                if mode == 'Mutation':
                    output_subplot.add_patch(
                        patches.FancyArrowPatch(
                            (0.81, 1.5*a),
                            (0.81, a/2),
                            connectionstyle = 'arc3, rad = -0.7',
                            mutation_scale = 20,
                            color = 'green'
                        )
                    )
                        
                if mode == 'Crossover':
                    output_subplot.add_patch(
                        patches.FancyArrowPatch(
                            (0.81, 2.5*a),
                            (0.85, a/2),
                            connectionstyle = 'arc3, rad = -0.9',
                            mutation_scale = 20,
                            color = 'green'
                        )
                    )
                    
                    output_subplot.add_patch(
                        patches.FancyArrowPatch(
                            (0.81, 1.5*a),
                            (0.81, a/2),
                            connectionstyle = 'arc3, rad = -0.7',
                            mutation_scale = 20,
                            color = 'green'
                        )
                    )
                    
                
        output_plot.set_size_inches(18.5, 10.5, forward=True)
                
        output_plot.canvas.draw()           
        output_plot.show()

        plt.show()
        plt.draw_all()
            
    # -------------------------------------------------------------------------
    
    
    def draw_Q(self, Q):
        plt.plot(Q, 'ro-', color = 'green', label = 'min Q')

        plt.xlabel('Iteration')
        plt.ylabel('Q')
        plt.title('Q best on iteration')   
        
        plt.rcParams["figure.figsize"] = (12, 4)
        
        plt.show()
    
    # ------------------------------------------------------------------------- 
    
    def count_ones_in_bitmask(self, v):
        cnt = 0
        for k in range(len(v)):
            if v[k] == True:
                cnt+=1
        return cnt
    
    def calc_lower_envelope(self, R, Q):
        lower_envelope = [self.INF] * self.n
        
        for i in range(len(R)):
            j = self.count_ones_in_bitmask(R[i])     
            lower_envelope[j] = min(lower_envelope[j], Q[i])
            
        return lower_envelope
        
    def draw_lower_envelope(self, lower_envelope, index, right_bound):
        index += 1
        right_bound += 1
        
        x = []
        y = []
        
        for j in range(len(lower_envelope)):
            if lower_envelope[j] < self.INF:
                x.append(j)
                y.append(lower_envelope[j])
                
                
        marker = 'ro'
        if index == right_bound:
            marker = 'ro-'
                
        plt.plot(x, y, marker, 
                 color = ((right_bound-index)/right_bound, 
                          (right_bound-index)/right_bound, 
                          (right_bound-index)/right_bound
                         )
                )
                
        return
    
    def draw_p_difference(self, prev_p, p):
        ind = np.arange(len(p))    # the x locations for the groups
        width = 0.35 
        
        diff = np.array(p)-np.array(prev_p)
        diff *= -1
        
        diff1 = []
        diff2 = []
        
        for k in range(len(p)):
            diff1.append(max(diff[k], 0.0))
            diff2.append(min(diff[k], 0.0))
        
        p1 = plt.bar(ind, p, width)
        p2 = plt.bar(ind, diff1, width, bottom = p, color = 'red')
        p3 = plt.bar(ind, diff2, width, bottom = p, color = 'green')
        
        plt.ylabel('p')
        plt.xlabel('Feature')
        plt.title('Probability')
        
        plt.show()
        
    
    def fit(self, X_train, y_train, q_criterion, d, T, r, h, visualization):
        self.X_train = X_train
        self.y_train = y_train
        self.d = d
        self.T = T
        self.r = r
        self.h = h
        
        self.INF = 1E9
        
        rand_gen = random.Random()
        
        # n - number of features
        self.n = len(X_train[0])
        
        # init feature probabilities
        p = [1.0/self.n] * self.n
        prev_p = copy.deepcopy(p)
        
        Q_best_history = []
        
        lower_envelope_history = []
        
        # ---------------------------------------------------------------------
            
        best_F = []
        Q_best  = self.INF
     
        for t in range(T):
            if visualization == True:
                self.draw_p_difference(prev_p, p)
            
            R = []
            while (len(R) < r):
                obj = []
                for k in range(self.n):
                    flag = rand_gen.choices([True, False], weights = [p[k], 1-p[k]], k = 1)[0]
                    obj.append(flag)
                    
                if self.count_ones_in_bitmask(obj) > 0:
                    R.append(obj)
                        
            
            Q = []
            for i in range(len(R)):
                Q.append(q_criterion.calc_Q(R[i]))
                
            R = [x for _,x in sorted(zip(Q, R))]
            
            Q = []
            for i in range(len(R)):
                Q.append(q_criterion.calc_Q(R[i]))
                
            print('Best Q on iteration: ', Q[0])
            print('Best Q:', Q_best)
            
            if visualization == True:
                lower_envelope = self.calc_lower_envelope(R, Q)
                
                lower_envelope_history.append(lower_envelope)
                
                for k in range(len(lower_envelope_history)):
                    self.draw_lower_envelope(lower_envelope_history[k], index = k, 
                                             right_bound = t)
            
            
            plt.show()
        
            F_min = R[0]
            F_max = R[len(R)-1]
            
            H = 0.0
            
            prev_p = copy.deepcopy(p)
            
            for k in range(len(F_max)):
                if F_max[k] == True:
                    delta_p_k = min(p[k], h)
                    p[k] -= delta_p_k
                    H += delta_p_k
                    
            cnt_ones_in_F_min = self.count_ones_in_bitmask(F_min)
                    
            for k in range(len(F_min)):
                if F_min[k] == True:
                    p[k] += H/cnt_ones_in_F_min
            
            
            if Q[0] < Q_best:
                Q_best = copy.deepcopy(Q[0])
                best_F = R[0]
            
            Q_best_history.append(Q[0])
        
            print('Iteration:', t)
    
    
        print('Best Q:', q_criterion.calc_Q(best_F))
        if visualization == True:
            self.draw_Q(Q_best_history)
        return best_F
           
            
    # -------------------------------------------------------------------------
            
def print_feature_names(F, names_of_features):
    for i in range(len(F)):
        if F[i] == 1:
            print(names_of_features[i], end = ' ')
    print()
    
        
def main():
    dp = data_preprocessor.Data_Preprocessor()
    X_train, y_train, X_test, y_test, names_of_features = dp.Prepare_data('diabetes')
    
    q_criterion = Q_Criterion(X_train, y_train, X_test, y_test)
    
    ss = Stochastic_Search()
    
    n = len(X_train[0])
    r = 500
    
    F = ss.fit(X_train, y_train, q_criterion, d = 1, T = 300, r = r, h = 1.0/(r * n), visualization = False)    
    
    print_feature_names(F, names_of_features)
    
    
main() 
        

Best Q on iteration:  3302.572149932065
Best Q: 1000000000.0
Iteration: 0
Best Q on iteration:  3067.1077629354622
Best Q: 3302.572149932065
Iteration: 1
Best Q on iteration:  3287.3262060002544
Best Q: 3067.1077629354622
Iteration: 2
Best Q on iteration:  3062.0681787697804
Best Q: 3067.1077629354622
Iteration: 3
Best Q on iteration:  3191.3196725972916
Best Q: 3062.0681787697804
Iteration: 4
Best Q on iteration:  3292.080215233633
Best Q: 3062.0681787697804
Iteration: 5
Best Q on iteration:  3122.5466749880725
Best Q: 3062.0681787697804
Iteration: 6
Best Q on iteration:  3269.9702105375577
Best Q: 3062.0681787697804
Iteration: 7
Best Q on iteration:  3307.8450317057905
Best Q: 3062.0681787697804
Iteration: 8
Best Q on iteration:  3302.572149932065
Best Q: 3062.0681787697804
Iteration: 9
Best Q on iteration:  3302.572149932065
Best Q: 3062.0681787697804
Iteration: 10
Best Q on iteration:  3150.400545556058
Best Q: 3062.0681787697804
Iteration: 11
Best Q on iteration:  3015.0009754829


Best Q on iteration:  3086.082768850167
Best Q: 3015.0009754829
Iteration: 105
Best Q on iteration:  3304.8112430770943
Best Q: 3015.0009754829
Iteration: 106
Best Q on iteration:  3150.400545556058
Best Q: 3015.0009754829
Iteration: 107
Best Q on iteration:  3302.572149932065
Best Q: 3015.0009754829
Iteration: 108
Best Q on iteration:  3248.065317194957
Best Q: 3015.0009754829
Iteration: 109
Best Q on iteration:  3258.8333689964834
Best Q: 3015.0009754829
Iteration: 110
Best Q on iteration:  3150.400545556058
Best Q: 3015.0009754829
Iteration: 111
Best Q on iteration:  3335.941115936342
Best Q: 3015.0009754829
Iteration: 112
Best Q on iteration:  3127.9709285701165
Best Q: 3015.0009754829
Iteration: 113
Best Q on iteration:  3147.524492649263
Best Q: 3015.0009754829
Iteration: 114
Best Q on iteration:  3079.3414198481505
Best Q: 3015.0009754829
Iteration: 115
Best Q on iteration:  3147.524492649263
Best Q: 3015.0009754829
Iteration: 116
Best Q on iteration:  3271.258633266258
Best Q: 

Best Q on iteration:  3079.3414198481505
Best Q: 2990.647916074318
Iteration: 208
Best Q on iteration:  3150.400545556058
Best Q: 2990.647916074318
Iteration: 209
Best Q on iteration:  3302.572149932065
Best Q: 2990.647916074318
Iteration: 210
Best Q on iteration:  3150.400545556058
Best Q: 2990.647916074318
Iteration: 211
Best Q on iteration:  3150.400545556058
Best Q: 2990.647916074318
Iteration: 212
Best Q on iteration:  3034.9587391075493
Best Q: 2990.647916074318
Iteration: 213
Best Q on iteration:  3146.1124430869513
Best Q: 2990.647916074318
Iteration: 214
Best Q on iteration:  3316.8807715942626
Best Q: 2990.647916074318
Iteration: 215
Best Q on iteration:  3150.400545556058
Best Q: 2990.647916074318
Iteration: 216
Best Q on iteration:  3112.328124068183
Best Q: 2990.647916074318
Iteration: 217
Best Q on iteration:  3028.4679972620197
Best Q: 2990.647916074318
Iteration: 218
Best Q on iteration:  3062.0681787697804
Best Q: 2990.647916074318
Iteration: 219
Best Q on iteration:  